In [89]:
import statsmodels
from statsmodels import stats
from statsmodels.stats import weightstats
from scipy.stats import t,norm
import numpy as np
import pandas as pd
import  statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats import proportion
from scipy.stats import chisquare,chi2_contingency
import scipy
from scipy import stats
import pylab
from pylab import plot,show,legend,title,xlabel,ylabel
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from mord import LogisticAT

# Ordinal Logistic Regression

In [90]:
pip install mord

Note: you may need to restart the kernel to use updated packages.


In [91]:
df = pd.read_excel("CDAC_DataBook.xlsx",sheet_name='ordinal')

In [92]:
reg_dummy = pd.get_dummies(df['Region'],drop_first=True).astype(int)
df=df.drop(['Region'],axis=1)
df = pd.concat([df,reg_dummy],axis=1)

In [93]:
x_train,x_test,y_train,y_test = train_test_split(df.drop('Survival',axis=1),df.Survival,test_size=0.2)

In [94]:
x_train = sm.add_constant(x_train,prepend = False)
x_test = sm.add_constant(x_test,prepend = False)

In [95]:
mod3 = LogisticAT().fit(x_train,y_train)

In [15]:
x_train.head()

,ToxicLevel,2,const
3,32.0,1,1.0
15,56.0,0,1.0
57,38.5,1,1.0
55,42.0,1,1.0
36,40.0,1,1.0


## When toxicity level increase in the same region survival rate changes 

In [18]:
mydata = pd.DataFrame([[20,0,1],[40,0,1],[60,0,1],[80,0,1]])
mydata.columns = x_train.columns

In [19]:
mod3.predict(mydata)

array([3, 2, 1, 1])

In [20]:
mydata1 = pd.DataFrame([[20,1,1],[40,1,1],[60,1,1],[80,1,1]])
mydata1.columns = x_train.columns
mod3.predict(mydata1)

array([3, 2, 1, 1])

## When region changes survival rating does not change (Null Hypothesis)

In [22]:
mydata2 = pd.DataFrame([[20,0,1],[20,1,1],[40,0,1],[40,1,1],[60,0,1],[60,1,1]])
mydata2.columns = x_train.columns
mod3.predict(mydata2)

array([3, 3, 2, 2, 1, 1])

# Count Models

In [23]:
# Poisson and Negative Binomial when discrete data

In [27]:
#For possion when numbers are discrete but not as much variation for respones 
# Example of number of placements for our batch(maths marks,no_placement,program), this situation is called underdisperesed situation 
#(difference in maximum and minimum is v little) we use poission
# When variation of maximum and minimum is high we call it overdispersed then we use negative binomial

In [31]:
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import Poisson as psn
from statsmodels.discrete.discrete_model import NegativeBinomial as nbm

In [33]:
df = pd.read_excel('CDAC_DataBook.xlsx',sheet_name='poisson')
df.head()

,num_awards,prog,math
0,0,3,41
1,0,1,41
2,0,3,44
3,0,3,42
4,0,3,40


In [34]:
prog_dummy = pd.get_dummies(df['prog'],drop_first=True).astype(int)
df=df.drop(['prog'],axis=1)
df = pd.concat([df,prog_dummy],axis=1)

In [35]:
x_train,x_test,y_train,y_test = train_test_split(df.drop('num_awards',axis=1),df.num_awards,test_size=0.2)
x_train=sm.add_constant(x_train,prepend=False)
x_test=sm.add_constant(x_test, prepend=False)

In [36]:
df_train = pd.concat([x_train,y_train],axis=1)
df_test = pd.concat([x_test,y_test],axis=1)

In [37]:
df_train.columns = ['math','prog2','prog3','const','num_awards']
mod4 = psn.from_formula('num_awards~math+prog2+prog3', data=df_train).fit()

Optimization terminated successfully.
         Current function value: 0.866435
         Iterations 7


In [38]:
print(mod4.summary())

                          Poisson Regression Results                          
Dep. Variable:             num_awards   No. Observations:                  160
Model:                        Poisson   Df Residuals:                      156
Method:                           MLE   Df Model:                            3
Date:                Mon, 10 Jun 2024   Pseudo R-squ.:                  0.2060
Time:                        11:25:45   Log-Likelihood:                -138.63
converged:                       True   LL-Null:                       -174.59
Covariance Type:            nonrobust   LLR p-value:                 1.659e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.5281      0.779     -7.098      0.000      -7.055      -4.002
math           0.0704      0.012      5.725      0.000       0.046       0.094
prog2          1.3230      0.433      3.055      0.0

In [45]:
15*0.0704

1.056

In [46]:
#The above number indicates when your program doesnt change (Prog1) as your math marks 
#increases for every 15marks your probability for getting an award increase

In [47]:
#Because prog3's interval has a chance of 0 existing in between and also because prog3's P>0.118 Null hypothesis doesn't get rejected

In [48]:
#Null Hypothese for math and prog2 get rejected

## Calculating how many num of awards you get when your math marks are 70

In [56]:
mydata3=pd.DataFrame([[70,0,0,1],[70,1,0,1],[70,0,1,1]], columns = ['math','prog2','prog3','const'])
mydata3

,math,prog2,prog3,const
0,70,0,0,1
1,70,1,0,1
2,70,0,1,1


In [57]:
mod4.predict(mydata3)

0    0.548455
1    2.059365
2    1.213438
dtype: float64

In [71]:
# We conclude that prog2 has highest chances to get awards more than prog 3

## Calculating minimum marks you need for program 2 to get an award

In [70]:
mydata4=pd.DataFrame([[20,1,0,1],[30,1,0,1],[40,1,0,1],[50,1,0,1],[55,1,0,1],[59.74,1,0,1],[60,1,0,1]], columns = ['math','prog2','prog3','const'])
mod4.predict(mydata4)

0    0.060982
1    0.123284
2    0.249238
3    0.503873
4    0.716430
5    1.000181
6    1.018655
dtype: float64

# Negative Binominal 

In [83]:
df = pd.read_excel('CDAC_DataBook.xlsx',sheet_name='neg_bin')

In [84]:
prog_dummy = pd.get_dummies(df['prog'],drop_first=True)
df=df.drop(['prog'],axis=1)
df = pd.concat([df,prog_dummy],axis=1)

In [85]:
x_train,x_test,y_train,y_test = train_test_split(df.drop('daysabs',axis=1),df.daysabs,test_size=0.2)
x_train=sm.add_constant(x_train,prepend=False)
x_test=sm.add_constant(x_test, prepend=False)

In [86]:
df_train = pd.concat([x_train,y_train],axis=1)
df_test = pd.concat([x_test,y_test],axis=1)

In [88]:
mod4 = nbm.from_formula('daysabs ~ math + General + Vocational',data = df_train).fit()
print(mod4.summary())

Optimization terminated successfully.
         Current function value: 2.755545
         Iterations: 21
         Function evaluations: 24
         Gradient evaluations: 24
                     NegativeBinomial Regression Results                      
Dep. Variable:                daysabs   No. Observations:                  251
Model:               NegativeBinomial   Df Residuals:                      247
Method:                           MLE   Df Model:                            3
Date:                Mon, 10 Jun 2024   Pseudo R-squ.:                 0.02827
Time:                        12:21:41   Log-Likelihood:                -691.64
converged:                       True   LL-Null:                       -711.77
Covariance Type:            nonrobust   LLR p-value:                 9.439e-09
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             